# Preparing input for MT5 model

In first attempt using subtitles in srt format
from the  webpage https://www.opensubtitles.com/en/tvshows/2009-lie-to-me/seasons/2/episodes/1-lie-to-me-the-core-of-it and the tv serie is " Lie to me"
Though even preparing small set something like 800 sentences were quite hard as people who were translating were splitting part of sentences in slightly different timestamps not mentioning style of translating. In the mean time looking for other sources I found  https://github.com/Helsinki-NLP/Tatoeba-Challenge/blob/master/data/README-v2020-07-28.md 
and vast qantity of almost ready to use resources like big file below:

https://object.pouta.csc.fi/Tatoeba-Challenge-v2020-07-28/eng-pol.tar




## 1. Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Getting data sets  from internet

In [ ]:
!wget -P /content/drive/MyDrive/mt5/ https://object.pouta.csc.fi/Tatoeba-Challenge-v2020-07-28/eng-pol.tar 

--2021-04-21 16:15:33--  https://object.pouta.csc.fi/Tatoeba-Challenge-v2020-07-28/eng-pol.tar
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3580375040 (3.3G) [application/x-tar]
Saving to: ‘/content/drive/MyDrive/mt5/eng-pol.tar’

eng-pol.tar         100%[===================>]   3.33G  33.8MB/s    in 1m 57s  

2021-04-21 16:17:31 (29.2 MB/s) - ‘/content/drive/MyDrive/mt5/eng-pol.tar’ saved [3580375040/3580375040]



## 3. Extracting from tar

In [ ]:
!tar -xvf /content/drive/MyDrive/mt5/eng-pol.tar -C /content/drive/MyDrive/mt5

data/release/v2020-07-28/eng-pol/
data/release/v2020-07-28/eng-pol/train.src.gz
data/release/v2020-07-28/eng-pol/dev.trg
data/release/v2020-07-28/eng-pol/train.id.gz
data/release/v2020-07-28/eng-pol/test.trg
data/release/v2020-07-28/eng-pol/langids
data/release/v2020-07-28/eng-pol/test.id
data/release/v2020-07-28/eng-pol/dev.src
data/release/v2020-07-28/eng-pol/dev.id
data/release/v2020-07-28/eng-pol/README.md
data/release/v2020-07-28/eng-pol/test.src
data/release/v2020-07-28/eng-pol/train.trg.gz


### unzipping

In [ ]:
!gzip -dk /content/drive/MyDrive/mt5/data/release/v2020-07-28/eng-pol/train.src.gz

In [ ]:
!gzip -dk /content/drive/MyDrive/mt5/data/release/v2020-07-28/eng-pol/train.trg.gz

## Taking 5% of training file

after taking out original files their size(~3GB) cause crasking colab session.
To go forward I was gradually decreasing size of input file to ~5% which was 1 mln  lines ( senteces)

In [ ]:
import os
import pandas as pd

In [ ]:
def take_some_lines(filename_in,filename_out, nr_lines):
  input_file = open(filename_in,'r')
  output_file = open(filename_out,'w')

  for lines in range(nr_lines):
      line = input_file.readline()
      output_file.write(line)

In [ ]:
take_some_lines('/content/drive/MyDrive/mt5/data/eng-pol/train.src','/content/drive/MyDrive/mt5/data/eng-pol/train_1e06.src', 1000000)

In [ ]:
take_some_lines('/content/drive/MyDrive/mt5/data/eng-pol/train.trg','/content/drive/MyDrive/mt5/data/eng-pol/train_1e06.trg', 1000000)

In [ ]:
def prepare_translation_datasets(filename_src, filename_trg,data_path):
    with open(os.path.join(data_path, filename_trg), "r", encoding="utf-8") as f:
        polish_text = f.readlines()
        polish_text = [text.strip("\n") for text in polish_text]
        
        print('target is there')

    with open(os.path.join(data_path, filename_src), "r") as f:
        english_text = f.readlines()
        english_text = [text.strip("\n") for text in english_text]
        
        print('source is there')

    data = []
    for polish, english in zip(polish_text, english_text):
        data.append(["translate polish to english", polish, english])
        data.append(["translate english to polish", english, polish])

    df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])
    return df

In [ ]:
train_df = prepare_translation_datasets("train_1e06.src","train_1e06.trg","/content/drive/MyDrive/mt5/data/eng-pol")

target is there
source is there


In [ ]:
train_df.info

<bound method DataFrame.info of                               prefix  ...                                        target_text
0        translate polish to english  ...                            - You could've kept it.
1        translate english to polish  ...                             - Mogłeś go zatrzymać.
2        translate polish to english  ...  Okay, even if any of this was remotely legitim...
3        translate english to polish  ...  Przyjmując, że to jest chociaż odrobinę wiaryg...
4        translate polish to english  ...  Strong stakeholder participation, especially i...
...                              ...  ...                                                ...
1999995  translate english to polish  ...  Mężczyźni nie tylko utrzymywali niedozwolone s...
1999996  translate polish to english  ...  Over the next decade, the transformative impac...
1999997  translate english to polish  ...  W ciągu następnego dziesięciolecia prowadzące ...
1999998  translate polish to english  

In [ ]:
train_df.sample(10)

,prefix,input_text,target_text
820815,translate english to polish,Don't nobody wanna let us have ours.,Nikt nie pozwala nam mieć nic swojego.
289567,translate english to polish,There he said with reference to Jesus ’ transf...,"Nawiązując do przemienienia Jezusa, powiedział..."
1651949,translate english to polish,Between 1945 and 1949 she was working in the M...,Pomiędzy 1945 a 1949 pracowała w Ministerstwie...
1924796,translate polish to english,Kto nasłał na mnie tego gościa na lotnisku?,Who sent that cat after me at the airport? And...
772556,translate polish to english,No proszę!,"- That's huge. - It is, yeah."
1884787,translate english to polish,'I'm alive.',/Żyję.
678538,translate polish to english,"Słuchaj, mógłbym się z tego wymiksować, gdyby ...","Look, I could've played my way out of that non..."
1255162,translate polish to english,A niech mnie...,"Well, I'll be a son of a..."
230955,translate english to polish,Forest Strategy: http://ec.europa.eu/agricultu...,Strategię leśną UE: http://ec.europa.eu/agricu...
1556987,translate english to polish,You can't just put it in there?,Sam nie możesz włożyć?


In [ ]:
eval_df = prepare_translation_datasets("test.src","test.trg","/content/drive/MyDrive/mt5/data/eng-pol")

target is there
source is there


In [ ]:
eval_df.sample(10)

,prefix,input_text,target_text
5228,translate polish to english,Nie ruszaj się.,Don't move.
10720,translate polish to english,"Nigdy nie było dobrej wojny, ani złego pokoju.",There never was a good war nor a bad peace.
1880,translate polish to english,"Wróciłem, żeby pogadać z tobą.",I came back to talk to you.
10058,translate polish to english,Ja płacę.,I'm paying.
8208,translate polish to english,Obaj są w pokoju.,Both of them are in the room.
7283,translate english to polish,"A beer, please.",Poproszę piwo.
12179,translate english to polish,It was a moving sight.,To był poruszający widok.
12972,translate polish to english,Miasta są większe od wsi.,Towns are larger than villages.
8840,translate polish to english,Chcę być niezależny.,I want to be independent.
18784,translate polish to english,Kiedyś zostanę lekarzem.,I will be a doctor someday.


In [ ]:
train_df.to_csv("/content/drive/MyDrive/mt5/data/eng-pol/train.tsv", sep="\t")
eval_df.to_csv("/content/drive/MyDrive/mt5/data/eng-pol/eval.tsv", sep="\t")

lets check how may lines has training file

In [ ]:
with open('/content/drive/MyDrive/mt5/data/eng-pol/train.src') as my_file:
    print(sum(1 for _ in my_file))

59742979


In [ ]:
with open('/content/drive/MyDrive/mt5/data/eng-pol/train.trg') as my_file:
    print(sum(1 for _ in my_file))

59742979


In [ ]:
!head /content/drive/MyDrive/mt5/data/eng-pol/train.src

- You could've kept it.
Okay, even if any of this was remotely legitimate, why us?
Strong stakeholder participation, especially industry involvement and RG cooperation
You think I want to do this?
He's been raped."
(13 votes) Level: laptop laptop laptop Users think this product is suitable for e-merchants who are: Beginners 10 Intermediate 1 Advanced 2 +100downloads!
It was hard on Emily having a brother like Owen.
[ Picture on page 10]
So I guess we have the wholeinsurance thing worked out.
The action also aims to address the gap in institutional communication on social networks in local languages.


In [ ]:
!head /content/drive/MyDrive/mt5/data/eng-pol/train.trg

- Mogłeś go zatrzymać.
Przyjmując, że to jest chociaż odrobinę wiarygodne: dlaczego my?
znaczny udział zainteresowanych stron, przede wszystkim zaangażowanie sektora i współpraca ze strony zrzeszenia badawczego;
Myślisz, że tego chcę?
Został zgwałcony?"
(13 głosy) Poziom kompetencji laptop laptop laptop Użytkownicy oceniają że wymagany poziom kompetencji do tego produktu jest: Dla początkujących użytkowników 10 Dla średniozaawansowanych użytkowników 1 Dla zaawansowanych użytkowników 2 +100downloads!
Ciężko było Emily mieć takiego brata jak Owen.
[ Ilustracja na stronie 10]
Więc mamy chyba mamy już za sobą tą całą sprawę z ubezpieczeniem.
Miejsce realizacji
